In [1]:
! pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00


In [2]:
!pip install numpy==1.24

In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
import accelerate
accelerate.__version__

'0.28.0'

In [17]:
from huggingface_hub import notebook_login

notebook_login()

## Load dataset

In [5]:
import pandas as pd
from datasets import Dataset

In [6]:
sents = pd.read_csv('/content/sentences.tsv', sep = '\t')
sents.columns=['text']
sents = Dataset.from_pandas(sents)

In [7]:
sents = sents.train_test_split(test_size=0.2)

In [8]:
sents["train"][0]

{'text': 'They would like to achive as much as they could, but sometimes directly these people are suffering from a huge amount of health problems, deadly desises, mental problems etc.'}

## Preprocess

In [31]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [32]:
tokenizer.pad_token = tokenizer.eos_token

In [33]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [34]:
tokenized_sents = sents.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=sents["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/10745 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2687 [00:00<?, ? examples/s]

In [35]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## Train

In [36]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [37]:
training_args = TrainingArguments(
    output_dir="GPT2_L1_sent_generator",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sents["train"],
    eval_dataset=tokenized_sents["test"],
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.907600,3.723125
2,3.620300,3.624157
3,3.494900,3.597818


TrainOutput(global_step=4032, training_loss=3.705113312554738, metrics={'train_runtime': 1289.7643, 'train_samples_per_second': 24.993, 'train_steps_per_second': 3.126, 'total_flos': 1837463747328000.0, 'train_loss': 3.705113312554738, 'epoch': 3.0})

In [38]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [39]:
trainer.push_to_hub()

events.out.tfevents.1712273979.c81065ef0388.383.3:   0%|          | 0.00/359 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1712272662.c81065ef0388.383.2:   0%|          | 0.00/7.69k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/GPT2_L1_sent_generator/commit/b4244f85079f18a772c7596e7137c097a424ccbc', commit_message='End of training', commit_description='', oid='b4244f85079f18a772c7596e7137c097a424ccbc', pr_url=None, pr_revision=None, pr_num=None)

In [40]:
trainer.model.push_to_hub("Zlovoblachko/GPT2_L1_sent_generator")

README.md:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/GPT2_L1_sent_generator/commit/e436313c46fe5040e9cfa169979037587ef08410', commit_message='Upload model', commit_description='', oid='e436313c46fe5040e9cfa169979037587ef08410', pr_url=None, pr_revision=None, pr_num=None)

In [41]:
tokenizer.push_to_hub("Zlovoblachko/GPT2_L1_sent_generator")

README.md:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/GPT2_L1_sent_generator/commit/b99462eb511c29df306b5794f5b32a6299090e90', commit_message='Upload tokenizer', commit_description='', oid='b99462eb511c29df306b5794f5b32a6299090e90', pr_url=None, pr_revision=None, pr_num=None)

## Inference

In [21]:
prompt = "It can be seen"

In [42]:
from transformers import pipeline

generator = pipeline("text-generation", model="Zlovoblachko/GPT2_L1_sent_generator")

config.json:   0%|          | 0.00/924 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

In [43]:
generator(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'It can be seen from this chart that, in 2010 the percentage of people were 61% and 57% in 1995, but when comparing to this year the proportion of aged people in Italy was stable at 56% and 57% in 2000. The percentage'}]